In [1]:
import aocd
from aocd.models import Puzzle
from aocd import submit
import re
import numpy as np
from tqdm import tqdm

In [2]:
current_day = 15
current_year = 2022
puzzle = Puzzle(year=current_year, day=current_day)
puzzle

<Puzzle(2022, 15) at 0x7f117223b220 - Beacon Exclusion Zone>

## Part 1



In [3]:
test_input = '''Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3'''

In [4]:
test = 0

if test:
    input_data = test_input
    target_y = 10
    min_coord = 0
    max_coord = 20
else:
    input_data = puzzle.input_data
    target_y = 2_000_000
    min_coord = 0
    max_coord = 4_000_000

In [5]:
%%time
x_candidates = set()
for line in input_data.splitlines():
    sensor_x,sensor_y,beacon_x,beacon_y = list(map(int,re.findall("-?\d+", line)))
    man_dist = abs(sensor_x-beacon_x) + abs(sensor_y-beacon_y)
    man_dist_x = man_dist - abs(target_y-sensor_y)
    if man_dist_x < 0:
        continue
    x_candidates.update(list(range(sensor_x - man_dist_x, sensor_x + man_dist_x + 1)))
    if beacon_y == target_y:
        x_candidates.difference_update([beacon_x])
result = len(x_candidates)
result

CPU times: user 233 ms, sys: 199 ms, total: 432 ms
Wall time: 431 ms


6425133

In [6]:
puzzle.answer_a = result
puzzle.answer_a

'6425133'

## Part 2



In [7]:
def union_ranges(ranges):
    ranges = sorted(ranges)
    # merge the ranges
    low_x, high_x = ranges[0]
    output_ranges = set()
    for x1,x2 in ranges[1:]:
        if x1 <= high_x:
            high_x = max(high_x, x2)
        else:
            output_ranges.add((low_x, high_x))
            low_x, high_x = x1, x2

    output_ranges.add((low_x, high_x))
    output_ranges = sorted(list(output_ranges))
    
    return output_ranges

In [8]:
data = np.array([list(map(int,re.findall("-?\d+", line))) for line in input_data.splitlines()])
data = np.hstack((data, (np.abs(data[:,0] - data[:,2]) + np.abs(data[:,1] - data[:,3])).reshape((-1,1))))

for y in tqdm(range(0,max_coord+1)):
    man_dist_remaining = data[:,4] - np.abs(data[:,1] - y)
    flag = man_dist_remaining>=0
    ranges = np.vstack((data[flag,0] - man_dist_remaining[flag], data[flag,0] + man_dist_remaining[flag])).transpose()
    ranges = np.minimum(np.maximum(ranges,[min_coord,min_coord]),[max_coord,max_coord])
    ranges = [list(row) for row in ranges]
    output_ranges = union_ranges(ranges)
    if len(output_ranges) == 1: # and output_ranges[0][0] <= min_coord and output_ranges[0][1] >= max_coord:
        continue
    assert len(output_ranges) == 2
    x1, x2 = output_ranges[0]
    x3, x4 = output_ranges[1]
    assert x3 - x2 == 2 # only one item between the two ranges
    possible_distress_beacon = x2 + 1, y
    result = 4000000 * possible_distress_beacon[0] + possible_distress_beacon[1]
    print(result)
    break

 86%|████████▌ | 3429555/4000001 [01:06<00:11, 51786.68it/s]

10996191429555


In [9]:
print(possible_distress_beacon)

(2749047, 3429555)


In [10]:
puzzle.answer_b = result
puzzle.answer_b

Part b already solved with same answer: 10996191429555


'10996191429555'